In [62]:
import requests
import pandas as pd
from io import BytesIO
import os
from io import StringIO

In [ ]:
# https://www.dataestur.es/en/apidata/

In [92]:
# Añadimos las urls de las que necesitamos descargar archivos. Key = Name; Value = URL

downloads = {
    "FRONTUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/FRONTUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&Tipo%20de%20visitante=Todos&CCAA%20de%20destino=Todos",
    "EGATUR_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EGATUR_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs%20de%20residencia=Todos&CCAA%20de%20destino=Todos",
    "ACTIVIDADES_OCIO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ACTIVIDADES_OCIO_DL?CCAA=Todos&Provincia=Todos",
    "TRANSPORTE_TERRESTRE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/TRANSPORTE_TERRESTRE_DL?desde%20%28a%C3%B1o%29=2018&Tipo%20transporte=Todos",
    "CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL?Pa%C3%ADs%20origen=Todos&Ciudad%20destino=Todos&Tipo%20origen=Todos",
    "ESCUCHA_ACTIVA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/ESCUCHA_ACTIVA_DL?Idioma=Todos&Medio=Todos&Sentimiento=Todos",
    "CST_GASTO_CONSUMO_TURISTICO_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/CST_GASTO_CONSUMO_TURISTICO_DL?Producto=Todos",
    "EOAC_CATEGORIA_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EOAC_CATEGORIA_DL?desde%20%28a%C3%B1o%29=2018&Categor%C3%ADa=Todos&Lugar%20de%20residencia=Todos",
    "EOAL_PAIS_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EOAL_PAIS_DL?Pa%C3%ADs=Todos&Lugar%20de%20residencia=Todos",
    "EOTR_PAIS_DL": "https://dataestur.azure-api.net/API-SEGITTUR-v1/EOTR_PAIS_DL?desde%20%28a%C3%B1o%29=2018&Pa%C3%ADs=Todos&Lugar%20de%20residencia=Todos"
}

In [95]:
# Creamos una función para descargar un diccionario de URLS. Para ello tenemos que pasarle la url y un nombre de archivo.
# La descarga devuelve un tipo binario, así que usamos BytesIO y lo convertimos en Dataframe con Pandas.

def download_file(url, file_name, folder="./data"):
    file_path = os.path.join(folder, f"{file_name}.xlsx")

    if os.path.exists(file_path):
        print(f"- {file_name}.xlsx already exists.")
        return pd.read_excel(file_path)

    response = requests.get(url)
    
    if response.status_code == 200:
        file_type = response.headers["Content-Disposition"].split(".")[-1]

        if file_type == "xlsx":
            df = pd.read_excel(BytesIO(response.content))
            df.to_excel(file_path)
            print(f"{file_name}.xlsx downloaded and saved.")
            return df
        elif file_type == "csv":
            encoding = response.apparent_encoding
            df = pd.read_csv(StringIO(response.content.decode(encoding)), sep=';', engine='python')
            df.to_excel(file_path)
            print(f"{file_name}.csv downloaded and saved.")
            return df
    else:
        print(f"Failed to download {file_name}: {response.status_code}")
        return None

In [96]:
# Guardamos todos los dataframes. Llamamos nuestra función para crear un nuevo diccionario de dataframes.

datasets = {}

for name, url in downloads.items():
    df = download_file(url, name)
    if df is not None:
        datasets[name] = df

- FRONTUR_DL.xlsx already exists.
- EGATUR_DL.xlsx already exists.
- ACTIVIDADES_OCIO_DL.xlsx already exists.
- TRANSPORTE_TERRESTRE_DL.xlsx already exists.
- CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL.xlsx already exists.
- ESCUCHA_ACTIVA_DL.xlsx already exists.
- CST_GASTO_CONSUMO_TURISTICO_DL.xlsx already exists.
- EOAC_CATEGORIA_DL.xlsx already exists.
- EOAL_PAIS_DL.xlsx already exists.
- EOTR_PAIS_DL.xlsx already exists.


In [84]:
#Para visualizar datos en conjunto:

for name, df in datasets.items():
    print(f"\n {name}")
    print("Columns:", df.columns.tolist())


 FRONTUR_DL
Columns: ['AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE', 'CCAA_DESTINO', 'VISITANTES']

 EGATUR_DL
Columns: ['AÑO', 'MES', 'PAIS_RESIDENCIA', 'TIPO_VISITANTE', 'CCAA_DESTINO', 'PERNOCTACIONES', 'GASTO_TOTAL']

 ACTIVIDADES_OCIO_DL
Columns: ['AÑO', 'MES', 'PRODUCTO', 'CATEGORIA', 'CCAA', 'PROVINCIA', 'ENTRADAS', 'VISITAS_PAGINAS', 'GASTO_TOTAL', 'PRECIO_MEDIO_ENTRADA', 'TRANSACCIONES']

 TRANSPORTE_TERRESTRE_DL
Columns: ['AÑO', 'MES', 'TIPO_TRANSPORTE', 'VIAJEROS']

 CONECTIVIDAD_AEREA_BUSQUEDAS_INICIO_VIAJE_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'PAIS_ORIGEN', 'CIUDAD_DESTINO', 'TIPO_ORIGEN', 'BUSQUEDAS', 'ANTELACION', 'ESTANCIA_MEDIA', 'VARIACION_INTERANUAL_BUSQUEDAS']

 ESCUCHA_ACTIVA_DL
Columns: ['Unnamed: 0', 'AÑO', 'MES', 'MENCIONES']

 CST_GASTO_CONSUMO_TURISTICO_DL
Columns: ['Unnamed: 0', 'AÑO', 'PRODUCTO', 'NUM_GASTO_INTERIOR', 'NUM_GASTO_INTERNO', 'NUM_GASTO_EMISOR', 'NUM_CONSUMO_TURISTICO_INTERIOR', 'NUM_CONSUMO_OTROS_COMPONENTES_TURISTICOS', 'NUM_GASTO_REC